In [ ]:
# Student Name: Qadeer Hussain
# Student ID: C00270632
# Patient Profile Functions:
# Create
# Encrypt
# Search
# Decrypt
# Display

In [17]:
import mysql.connector
from cryptography.fernet import Fernet

In [3]:
# Generate and save this key securely
key = Fernet.generate_key()
cipher_suite = Fernet(key)

In [4]:
# Load your previously saved key
def load_key():
    return open('secure.key', 'rb').read()

In [5]:
# Encrypt the msg using a key, converting the message and returning the encrypted result.
def encrypt_message(message, key):
    return Fernet(key).encrypt(message.encode())

In [6]:
# Key will be stored used later for decryption
with open('secure.key', 'wb') as key_file:
    key_file.write(key)

In [7]:
# Function to format contact numbers to drop the 0
def format_contact_number(number):
    if number.startswith('0'):
        number = '+353' + number[1:]
    return number

In [8]:
# Function to insert to create new patient profile
def create_patient(name, dob, contact_number, email_address, home_address, next_of_kin_name, emergency_contact_number, next_of_kin_home_address, emergency_email_address):
    conn = mysql.connector.connect(
        user='root',
        password='Qadeerh03',
        host='127.0.0.1',
        database='ecms',
        port=3306
    )
    cursor = conn.cursor()
    key = load_key()
    try:
        # Format the contact number and the emergency contact number
        formatted_contact_number = format_contact_number(contact_number)
        formatted_emergency_contact_number = format_contact_number(emergency_contact_number)

        # Encrypt data such as (Contact Number, Home Address, Email Address, Next of Kin Name, Emergency Contact Number, Next of Kin Home Address
        encrypted_contact_number = encrypt_message(formatted_contact_number, key)
        encrypted_email_address = encrypt_message(email_address, key)
        encrypted_home_address = encrypt_message(home_address, key)
        encrypted_next_of_kin_name = encrypt_message(next_of_kin_name, key)
        encrypted_emergency_contact_number = encrypt_message(formatted_emergency_contact_number, key)
        encrypted_next_of_kin_home_address = encrypt_message(next_of_kin_home_address, key)
        encrypted_emergency_email_address = encrypt_message(emergency_email_address, key)

        # SQL query to insert a new record into the patient_profile table
        create_patient_profile = '''
        INSERT INTO patient_profile (Name, DOB, Contact_Number, Email_Address, Home_Address, Next_Of_Kin_Name, Emergency_Contact_Number, Next_Of_Kin_Home_Address, Emergency_Email_Address)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''
        cursor.execute(create_patient_profile, (name, dob, encrypted_contact_number, encrypted_email_address, encrypted_home_address, encrypted_next_of_kin_name, encrypted_emergency_contact_number, encrypted_next_of_kin_home_address, encrypted_emergency_email_address))
        # Commit to the database
        conn.commit()
        print("Patient record created.")
    # If creating the patient fails throw error
    except mysql.connector.Error as e:
        print("Error occurred:", e)
    finally:
        cursor.close()
        conn.close()

In [9]:
# Dummy Data
create_patient(
    name='John Doe',
    dob='2024-01-01',
    contact_number='0123456789',
    email_address='johndoe@gmail.com',
    home_address='1234 Street Dublin',
    next_of_kin_name='Jane Doe',
    emergency_contact_number='0987654321',
    next_of_kin_home_address='5678 Street Dublin',
    emergency_email_address='janedoe@gmail.com'
)

Patient record created.


In [10]:
# Dummy Data
create_patient(
    name='Joe Blogs',
    dob='2024-12-12',
    contact_number='0123456789',
    email_address='joeblogs@gmail.com',
    home_address='1234 Street Dublin',
    next_of_kin_name='Mary Blogs',
    emergency_contact_number='0987654321',
    next_of_kin_home_address='5678 Street Dublin',
    emergency_email_address='maryblogs@gmail.com'
)

Patient record created.


In [11]:
# Dummy Data
create_patient(
    name='Joe Blogs',
    dob='2023-01-01',
    contact_number='0987654321',
    email_address='joeblogs@gmail.com',
    home_address='5678 Street Dublin',
    next_of_kin_name='Mary Blogs',
    emergency_contact_number='0123456789',
    next_of_kin_home_address='1234 Street Dublin',
    emergency_email_address='maryblogs@gmail.com'
)

Patient record created.


In [33]:
def search_patient(name=None, dob=None):
    conn = mysql.connector.connect(
        user='root',
        password='Qadeerh03',
        host='127.0.0.1',
        database='ecms',
        port=3306
    )
    cursor = conn.cursor()

    query = "SELECT Name, DOB, Contact_Number, Email_Address, Home_Address, Next_Of_Kin_Name, Emergency_Contact_Number, Next_Of_Kin_Home_Address, Emergency_Email_Address FROM patient_profile WHERE "
    params = []

    # If a name is provided append it to the query
    if name:
        query += "Name = %s"
        params.append(name)
    # If a date of birth (DOB) is provided append it to the query
    if dob:
        if name:
            query += " AND "
        query += "DOB = %s"
        params.append(dob)

    cursor.execute(query, params)
    results = cursor.fetchall()

    cursor.close()
    conn.close()

    return results

In [34]:
# This decrypts the encrypted messages which is the patient data
def decrypt_message(encrypted_message, key):
    return Fernet(key).decrypt(encrypted_message).decode()

In [35]:
# This decrypts selected fields of encrypted patient data rows and returns them in readable form by looping through them.
def decrypt_patient_data(patient_data, key):
    decrypted_patient_data = []
    for data in patient_data:
        decrypted_row = [decrypt_message(column, key) if i > 1 else column for i, column in enumerate(data)]
        decrypted_patient_data.append(decrypted_row)
    return decrypted_patient_data

In [37]:
def display_patient_info():
    # User input for name and/or date of birth
    name = input("Enter patient's name: ").strip()
    dob = input("Enter patient's date of birth YYYY-MM-DD (press enter if unknown): ").strip()

    patient_data = search_patient(name, dob)

    # If no patient was found print the following statement
    if not patient_data:
        print("No patient found.")
        return

    key = load_key()
    # Decrypt the patient data using the loaded key
    decrypted_patient_data = decrypt_patient_data(patient_data, key)

    # Loop through each decrypted patient data and print them out
    for row in decrypted_patient_data:
        print("Name:", row[0])
        print("DOB:", row[1])
        print("Contact Number:", row[2])
        print("Email Address:", row[3])
        print("Home Address:", row[4])
        print("Next Of Kin Name:", row[5])
        print("Emergency Contact Number:", row[6])
        print("Next Of Kin Home Address:", row[7])
        print("Emergency Email Address:", row[8])
        print("")

display_patient_info()

Name: John Doe
DOB: 2024-01-01
Contact Number: +353123456789
Email Address: johndoe@gmail.com
Home Address: 1234 Street Dublin
Next Of Kin Name: Jane Doe
Emergency Contact Number: +353987654321
Next Of Kin Home Address: 5678 Street Dublin
Emergency Email Address: janedoe@gmail.com

